In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from scipy.stats.mstats import winsorize
import torch

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

import joblib

In [3]:
csv_path = r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\data\train_processed.csv"
embeddings_path = r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\cnn_features_efficientnetb3_torch.npy"


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🟢 Using device: {device}")

🟢 Using device: cuda


In [5]:
df = pd.read_csv(csv_path)
embeddings = np.load(embeddings_path)
print("Image Embeddings Shape:", embeddings.shape)

Image Embeddings Shape: (75000, 1536)


In [6]:
df['price'] = df['price'].replace(0, np.nan)
df.dropna(subset=['price'], inplace=True)
df = df.reset_index(drop=True)

In [7]:
X_img = embeddings[:len(df)]
y = df['price'].values

In [8]:
y_wins = winsorize(y, limits=[0.01, 0.01])  # cap top/bottom 1%
y_log = np.log1p(y_wins)


In [9]:
num_cols = ['value', 'quantity_standard', 'quantity_numeric', 'quantity_count', 'price_per_unit']
cat_cols = ['unit', 'unit_standard']

In [10]:
for c in num_cols:
    df[c] = df[c].fillna(df[c].median())

for c in cat_cols:
    df[c] = df[c].fillna("missing")

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
])

X_tab = preprocessor.fit_transform(df)
X_final = np.hstack([X_tab, X_img])

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_final, y_log, test_size=0.2, random_state=42)


In [12]:
def smape(y_true, y_pred):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [50]:
models = {
    "CatBoost": CatBoostRegressor(
        iterations=1500,
        learning_rate=0.05,
        depth=8,
        verbose=200,
        task_type="GPU" if device == "cuda" else "CPU"
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=1500,
        learning_rate=0.05,
        num_leaves=64,
        device="gpu" if device == "cuda" else "cpu",
        gpu_platform_id=0,
        gpu_device_id=0
    ),
    "XGBoost": XGBRegressor(
        n_estimators=1500,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method="gpu_hist" if device == "cuda" else "hist",
        predictor="gpu_predictor" if device == "cuda" else "auto"
    )
}

In [51]:

results = {}

In [52]:
for name, model in models.items():
    print(f"\n🚀 Training {name} on {device.upper()} ...")
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    smape_val = smape(y_val, y_val_pred)
    results[name] = smape_val
    print(f"✅ {name} SMAPE: {smape_val:.4f}")
    joblib.dump(model, f"{name}_model_gpu.pkl")


🚀 Training CatBoost on CUDA ...
0:	learn: 0.9033847	total: 197ms	remaining: 4m 55s
200:	learn: 0.3902060	total: 12.8s	remaining: 1m 22s
400:	learn: 0.3481262	total: 25.5s	remaining: 1m 9s
600:	learn: 0.3140416	total: 38.7s	remaining: 57.9s
800:	learn: 0.2853737	total: 52.1s	remaining: 45.4s
1000:	learn: 0.2609919	total: 1m 5s	remaining: 32.5s
1200:	learn: 0.2387400	total: 1m 18s	remaining: 19.6s
1400:	learn: 0.2189311	total: 1m 32s	remaining: 6.51s
1499:	learn: 0.2098687	total: 1m 38s	remaining: 0us
✅ CatBoost SMAPE: 18.9487

🚀 Training LightGBM on CUDA ...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 392870
[LightGBM] [Info] Number of data points in the train set: 60000, number of used features: 1562
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3050 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been 

c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


✅ LightGBM SMAPE: 17.6616

🚀 Training XGBoost on CUDA ...


c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:59:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:59:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:2676: UserWarning: [15:04:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUD

✅ XGBoost SMAPE: 17.6873


In [53]:
print("\n🎯 Validation SMAPE Scores:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")


🎯 Validation SMAPE Scores:
CatBoost: 18.9487
LightGBM: 17.6616
XGBoost: 17.6873


In [54]:
# %% ---------------- ENSEMBLE ----------------
import joblib
import numpy as np

# Load trained models
cat = joblib.load(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\with image\CatBoost_model_gpu.pkl")
lgb = joblib.load(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\with image\LightGBM_model_gpu.pkl")
xgb = joblib.load(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\with image\XGBoost_model_gpu.pkl")

# Predict on validation set
cat_pred = cat.predict(X_val)
lgb_pred = lgb.predict(X_val)
xgb_pred = xgb.predict(X_val)

# Weighted ensemble (you can tune weights)
weights = np.array([0.2, 0.5, 0.3])  # CatBoost, LightGBM, XGBoost
ensemble_pred = (weights[0]*cat_pred + weights[1]*lgb_pred + weights[2]*xgb_pred)

# SMAPE calculation
def smape(y_true, y_pred):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

smape_ensemble = smape(y_val, ensemble_pred)
print(f"\n🧠 Ensemble SMAPE: {smape_ensemble:.4f}")


c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:2676: UserWarning: [15:17:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:



🧠 Ensemble SMAPE: 16.9157


In [55]:
best_smape, best_w = 999, None
for w1 in np.linspace(0,1,11):
    for w2 in np.linspace(0,1-w1,11):
        w3 = 1 - w1 - w2
        if w3 < 0: continue
        pred = w1*cat_pred + w2*lgb_pred + w3*xgb_pred
        s = smape(y_val, pred)
        if s < best_smape:
            best_smape, best_w = s, (w1, w2, w3)
print(f"🏆 Best Ensemble SMAPE: {best_smape:.4f} | Weights: Cat={best_w[0]:.2f}, LGBM={best_w[1]:.2f}, XGB={best_w[2]:.2f}")


🏆 Best Ensemble SMAPE: 16.8946 | Weights: Cat=0.20, LGBM=0.40, XGB=0.40


In [57]:
%pip install optuna


   ---------------------------------------- 0/4 [Mako]
   ---------------------------------------- 0/4 [Mako]
   ---------------------------------------- 0/4 [Mako]
   ---------------------------------------- 0/4 [Mako]
   ---------------------------------------- 0/4 [Mako]
   ---------------------------------------- 0/4 [Mako]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   -------------------- ------------------- 2/4 [alembic]
   ------------------------------ --------- 3/4 [optuna]
   ------------------------------ --------- 3/4 [optuna]
   ------------------------------

In [13]:
import optuna
import gc
import torch
import numpy as np
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🟢 Using device: {device}")

def smape(y_true, y_pred):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# ✅ Safer wrapper to clear memory
def safe_run(func, *args, **kwargs):
    try:
        result = func(*args, **kwargs)
        torch.cuda.empty_cache()
        gc.collect()
        return result
    except Exception as e:
        print("⚠️ Trial failed:", e)
        torch.cuda.empty_cache()
        gc.collect()
        return np.inf

# ⚙️ Lighter trials count
N_TRIALS = 10  # reduce if GPU memory is low
N_ESTIMATORS = 800

# -------------------- CATBOOST --------------------
def objective_cat(trial):
    params = {
        "iterations": N_ESTIMATORS,
        "learning_rate": trial.suggest_float("learning_rate", 0.03, 0.15),
        "depth": trial.suggest_int("depth", 5, 9),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 8),
        "task_type": "GPU" if device == "cuda" else "CPU",
        "verbose": 0
    }
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return smape(y_val, preds)

study_cat = optuna.create_study(direction="minimize", study_name="CatBoost_tune")
safe_run(study_cat.optimize, objective_cat, n_trials=N_TRIALS)
print("🏆 Best CatBoost:", study_cat.best_value, study_cat.best_params)

# -------------------- LIGHTGBM --------------------
def objective_lgb(trial):
    params = {
        "n_estimators": N_ESTIMATORS,
        "learning_rate": trial.suggest_float("learning_rate", 0.03, 0.15),
        "num_leaves": trial.suggest_int("num_leaves", 31, 128),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
        "device": "gpu" if device == "cuda" else "cpu"
    }
    model = LGBMRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return smape(y_val, preds)

study_lgb = optuna.create_study(direction="minimize", study_name="LGBM_tune")
safe_run(study_lgb.optimize, objective_lgb, n_trials=N_TRIALS)
print("🏆 Best LightGBM:", study_lgb.best_value, study_lgb.best_params)

# -------------------- XGBOOST --------------------
def objective_xgb(trial):
    params = {
        "n_estimators": N_ESTIMATORS,
        "learning_rate": trial.suggest_float("learning_rate", 0.03, 0.15),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
        "tree_method": "gpu_hist" if device == "cuda" else "hist",
        "predictor": "gpu_predictor" if device == "cuda" else "auto"
    }
    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return smape(y_val, preds)

study_xgb = optuna.create_study(direction="minimize", study_name="XGB_tune")
safe_run(study_xgb.optimize, objective_xgb, n_trials=N_TRIALS)
print("🏆 Best XGBoost:", study_xgb.best_value, study_xgb.best_params)


[I 2025-10-12 15:46:10,395] A new study created in memory with name: CatBoost_tune


🟢 Using device: cuda


[I 2025-10-12 15:47:17,097] Trial 0 finished with value: 19.771162520245536 and parameters: {'learning_rate': 0.06951389196827557, 'depth': 7, 'l2_leaf_reg': 1.6091973745205792}. Best is trial 0 with value: 19.771162520245536.
[I 2025-10-12 15:48:12,600] Trial 1 finished with value: 22.08689234608158 and parameters: {'learning_rate': 0.03485412251515338, 'depth': 5, 'l2_leaf_reg': 7.697870168012522}. Best is trial 0 with value: 19.771162520245536.
[I 2025-10-12 15:49:26,555] Trial 2 finished with value: 19.678556758089833 and parameters: {'learning_rate': 0.14251673787589214, 'depth': 7, 'l2_leaf_reg': 4.404814325124123}. Best is trial 2 with value: 19.678556758089833.
[I 2025-10-12 15:51:35,369] Trial 3 finished with value: 19.369258190733117 and parameters: {'learning_rate': 0.14408306938875157, 'depth': 9, 'l2_leaf_reg': 7.550852900538564}. Best is trial 3 with value: 19.369258190733117.
[I 2025-10-12 15:52:28,895] Trial 4 finished with value: 21.22480888041135 and parameters: {'lea

🏆 Best CatBoost: 19.101374243488916 {'learning_rate': 0.07215543957048741, 'depth': 9, 'l2_leaf_reg': 1.9803100834678617}
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 392870
[LightGBM] [Info] Number of data points in the train set: 60000, number of used features: 1562
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3050 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1542 dense feature groups (88.35 MB) transferred to GPU in 0.045709 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 2.738917
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-10-12 15:59:22,341] Trial 0 finished with value: 19.721005793767173 and parameters: {'learning_rate': 0.12061694477425103, 'num_leaves': 46, 'max_depth': 5, 'subsample': 0.8446321206574058, 'colsample_bytree': 0.7727826747104558}. Best is trial 0 with value: 19.721005793767173.


...

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 392870
[LightGBM] [Info] Number of data points in the train set: 60000, number of used features: 1562
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3050 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1542 dense feature groups (88.35 MB) transferred to GPU in 0.045051 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 2.738917
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-10-12 16:15:46,262] Trial 9 finished with value: 18.22860883540897 and parameters: {'learning_rate': 0.13049939082344847, 'num_leaves': 104, 'max_depth': 8, 'subsample': 0.7174775342371419, 'colsample_bytree': 0.9031057714003817}. Best is trial 6 with value: 17.57905229729365.
[I 2025-10-12 16:15:46,486] A new study created in memory with name: XGB_tune


🏆 Best LightGBM: 17.57905229729365 {'learning_rate': 0.08593601830567485, 'num_leaves': 123, 'max_depth': 9, 'subsample': 0.7922781425192877, 'colsample_bytree': 0.8733554301126074}


c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:15:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:15:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:2676: UserWarning: [16:17:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUD

⚠️ Trial failed: [16:18:03] C:\actions-runner\_work\xgboost\xgboost\src\common\device_vector.cu:23: Memory allocation error on worker 0: bad allocation: cudaErrorMemoryAllocation: out of memory
- Free memory: 0B
- Requested memory: 8GB

🏆 Best XGBoost: 19.12593186654775 {'learning_rate': 0.12379735363794514, 'max_depth': 7, 'subsample': 0.9867483537493592, 'colsample_bytree': 0.7648130292810089}
